# Read and Write into FTP using Spark (Pyspark)

In [24]:
# Init (Don't modify this code, it is mandatory for both Development and Production modes)
APPLICATION_NAME = "ftp_test"
import os, sys
def delPyc(pycFile):
    os.remove(pycFile + ".pyc") if os.path.exists(pycFile + ".pyc") else None
try: APPLICATION_HOME = os.environ["APPLICATION_HOME"]
except: APPLICATION_HOME = ".."
delPyc(APPLICATION_HOME + "/lib/SparkInit")
sys.path.append(os.path.abspath(APPLICATION_HOME + "/lib"))
from SparkInit import *  ## check the lib dir
from SparkFTP import *  ## check the lib dir

## initiate the SparkContext
try: spark
except: spark = None
try: sc
except: sc = None
sparkInit = SparkInit(spark, sc, APPLICATION_HOME, DEFAULT_APPLICATION_NAME=APPLICATION_NAME)
spark = sparkInit.spark
sc = sparkInit.sc

## Reading From FTP 

In [6]:

sparkFtp = SparkFTP(sc)
## FTP Connection 
sparkFtp.openFtpConnection("your_host", 21, "user", "password") ## Try to use your credentials 
## Creating DF using FTP csv (example of file my_example.csv)
ftpfilesI =  sparkFtp.scanFtpFiles("/", recursive=False, startsWith=None, contains=None, endsWith=".csv") # "/" is the path dir
for line in ftpfilesI:
    if line == "/my_example.csv":
            ftpDataframePath = line
            df_ftp = sparkFtp.ftpCsvFileToSparkDf(ftpDataframePath, isHeader=True, columns=None, columnDelimiter=',', lineDelimiter='\n')
            if df_ftp is None :
                raise Exception("File 'my_example.csv' is empty"+ftpDataframePath+".")   
            break 
    else : 
        raise Exception('No file found with name my_example.csv check if the file existe in network space')    


In [7]:
df_ftp.show()

+-----+--------------+--------------+
|Index|Height(Inches)|Weight(Pounds)|
+-----+--------------+--------------+
|    1|      65.78331|      112.9925|
|    2|      71.51521|      136.4873|
|    3|      69.39874|      153.0269|
|    4|       68.2166|      142.3354|
|    5|      67.78781|      144.2971|
|    6|      68.69784|      123.3024|
|    7|      69.80204|      141.4947|
|    8|      70.01472|      136.4623|
|    9|      67.90265|      112.3723|
|   10|      66.78236|      120.6672|
|   11|      66.48769|      127.4516|
|   12|      67.62333|       114.143|
|   13|      68.30248|      125.6107|
|   14|      67.11656|      122.4618|
|   15|      68.27967|      116.0866|
|   16|       71.0916|      139.9975|
|   17|        66.461|      129.5023|
|   18|      68.64927|      142.9733|
|   19|      71.23033|      137.9025|
|   20|      67.13118|      124.0449|
+-----+--------------+--------------+
only showing top 20 rows



## Writing into FTP 

In [23]:
## We will add a new column into our Df and we xill name it extra
## And then we will write this new Df in the file "my_example_enriched.py"

In [15]:
from pyspark.sql import functions as F
df_ftp_new = df_ftp.withColumn("extra",F.lit("100"))

In [16]:
## The new DF
df_ftp_new.show()

+-----+--------------+--------------+-----+
|Index|Height(Inches)|Weight(Pounds)|extra|
+-----+--------------+--------------+-----+
|    1|      65.78331|      112.9925|  100|
|    2|      71.51521|      136.4873|  100|
|    3|      69.39874|      153.0269|  100|
|    4|       68.2166|      142.3354|  100|
|    5|      67.78781|      144.2971|  100|
|    6|      68.69784|      123.3024|  100|
|    7|      69.80204|      141.4947|  100|
|    8|      70.01472|      136.4623|  100|
|    9|      67.90265|      112.3723|  100|
|   10|      66.78236|      120.6672|  100|
|   11|      66.48769|      127.4516|  100|
|   12|      67.62333|       114.143|  100|
|   13|      68.30248|      125.6107|  100|
|   14|      67.11656|      122.4618|  100|
|   15|      68.27967|      116.0866|  100|
|   16|       71.0916|      139.9975|  100|
|   17|        66.461|      129.5023|  100|
|   18|      68.64927|      142.9733|  100|
|   19|      71.23033|      137.9025|  100|
|   20|      67.13118|      124.

In [20]:
## Write code
sparkFtp.sparkDfToFtpCsvFile(df_ftp_new, "/my_example_enriched.csv", isHeader=True, columnDelimiter=',',lineDelimiter='\n')

## List the present files

In [21]:
## Creating DF using FTP csv (example of file my_example.csv)
ftp_files =  sparkFtp.scanFtpFiles("/", recursive=False, startsWith=None, contains=None, endsWith=".csv") # "/" is the path dir

In [22]:
ftp_files

['/my_example.csv', '/my_example_enriched.csv']